# Generating SWC Files

ShuTu was one of the two ways initially experimented for the BioImage 2019 brightfield challenge. This notebook demonstrates how to use ShuTu to generate SWC files, running on Colab. This notebook essentially recreates the SWC generating part of [the ShuTu tutorial](http://personal.psu.edu/dzj2/ShuTu/), on Colab.

This notebook uses ShuTu's sample data; it does not use Allen Institute data. This just proves that ShuTu can run on Colab.

## ShuTu documentation
- [ShuTu home page](http://personal.psu.edu/dzj2/ShuTu/), essentially a well-done short tutorial 
- [ShuTu pre-print overview on biorxiv](https://www.biorxiv.org/content/10.1101/226548v2.abstract), & [the PDF](https://www.biorxiv.org/content/biorxiv/early/2019/04/19/226548.full.pdf)
- [ShuTu on Github](https://github.com/tingzhao/ShuTu), [licensed GPL](https://github.com/tingzhao/ShuTu/blob/master/recipe-shutu/meta.yaml#L41)


## ShuTu is two tools in one
ShuTu has two separately usable components. 

1. ShuTu SWC generator is a CLI tool, which makes SWC files.
2. ShuTu SWC editor is a cross-platform GUI desktop application.

The SWC generator can be deployed on Colab. The SWC editor cannot; the editor is desktop application, not a Jupyter component. 



## Install ShuTu

This next cell is from the ShuTu tutorial and build.sh, combined into one cell specifically for install and build on Colab.

In [0]:
%%shell

# Get the Linux distro of ShuTu (took less than a minute to DL)
wget https://psu.box.com/shared/static/wszpq3r6hwgoxo8eq0vb1k82fwrabycl.zip
unzip wszpq3r6hwgoxo8eq0vb1k82fwrabycl.zip # creates ShuTuUbuntu directory

#set -m # needed to avoid error: bash: no job control in this shell
cd ShuTuUbuntu

##########
# This next part is from build.sh.
#
#export PATH=$downloadDir/local/bin:$PATH
cd mylib
make
cd ..
make


## Install test data from ShuTu tutorial

The real goal is to use ShuTu on the BioImage challenge dataset from The Allen. But a good sanity checker is to use the granule cell example data from the ShuTu tutorial, as described in section, ["Image processing and automated reconstruction"](http://personal.psu.edu/dzj2/ShuTu/), quoting:

>Download the image files from [Granule Cell 63X Start](https://psu.box.com/shared/static/f4gs13fs0y9ckrjjo78oid7hkv61k2wq.zip). Unzip in the home directory, which should create a directory granuleCell63XStart.

Specifically, the link behind "Granule Cell 63X Start" is: [https://psu.box.com/shared/static/f4gs13fs0y9ckrjjo78oid7hkv61k2wq.zip](https://psu.box.com/shared/static/f4gs13fs0y9ckrjjo78oid7hkv61k2wq.zip)

It takes about a minute to download the 1.3G granule cell sample data.

In [0]:
# This is a 1.3GB image stack for use in skeletonization (make a SWC file). 
# Downloads in less than 2 minutes
!wget https://psu.box.com/shared/static/f4gs13fs0y9ckrjjo78oid7hkv61k2wq.zip
!unzip f4gs13fs0y9ckrjjo78oid7hkv61k2wq.zip

Seems there are 520 *.tif files in the sample data set.

In [0]:
!ls /content/granuleCell63XStart/slides/*.tif | wc -l

520


## Generate the sample data's SWC file

This next cell assumes the above cell was run, creating the directory `/content/granuleCell63XStart/` used below, containing 4 tiff files that need to be processed.

This takes on the order of **ten minutes** (not precisely time yet)

In [0]:
%%shell
cd ShuTuUbuntu

mpirun --allow-run-as-root -n 1 ./createTiffStacksZeiss /content/granuleCell63XStart/ granule
mpirun --allow-run-as-root -n 1 processImages /content/granuleCell63XStart/
mpirun --allow-run-as-root -n 1 ./stitchTiles /content/granuleCell63XStart/
mpirun --allow-run-as-root -n 1 ./ShuTuAutoTrace /content/granuleCell63XStart/ ShuTu.Parameters.granule63XTest.dat # creates *.swc file

Creating tiff stacks from images and xml files in /content/granuleCell63XStart/ with finameCommon=granule
Number of tasks= 1 My rank= 0 Running on 8ddec874899d
Finding image slices with finames ending with _ORG.tif 
/content/granuleCell63XStart/
filenameSliceCommon=/content/granuleCell63XStart/slides/DG-151213-S1_63X_20percentOverlap_100617_redo_
nTiles=4 nz=130 zstart=6 zend=135
/content/granuleCell63XStart/granule-1.tif
/content/granuleCell63XStart/granule-2.tif
/content/granuleCell63XStart/granule-3.tif
/content/granuleCell63XStart/granule-4.tif
Bright field images
Number of tasks= 1 My rank= 0 Running on 8ddec874899d
ntiff=4 filenameCommon=/content/granuleCell63XStart/granule-
Image dimension=(1040 1388 129) type=0 number of channels=3
Color stack. Save projection to /content/granuleCell63XStart/granule-1.Proj.tif
Image dimension=(1040 1388 129) type=0 number of channels=3
Color stack. Save projection to /content/granuleCell63XStart/granule-4.Proj.tif
Image dimension=(1040 1388 129

The above cell should eventually end in:
```
Creating swc file...
Deleting small branches of length 1...Deleting connections to the deleted points...Deleting points...Deleted 125 points. npoints=3064 
Deleting small branches of length 2...Deleting connections to the deleted points...Deleting points...Deleted 36 points. npoints=3028 
Deleting small branches of length 3...Deleting connections to the deleted points...Deleting points...Deleted 21 points. npoints=3007 
Deleting small branches of length 4...Deleting connections to the deleted points...Deleting points...Deleted 22 points. npoints=2985 
Deleting small branches of length 5...Deleting connections to the deleted points...Deleting points...Deleted 11 points. npoints=2974 
Removing connected points if the total length is smaller than 20.000000 micron...Deleted 223 noisy points.
Delete isolated branches with length smaller than 20.000000 and leaf branch smaller than 5
Saving SWC to /content/granuleCell63XStart/granule-.auto.swc
There 48 segments 4 trees
```

## Test probe of file system



In [0]:
!pwd
!ls -l
!echo ------------
!ls /content/granuleCell63XStart/granule*

/content
total 1645644
-rw-r--r-- 1 root root 1389488667 Oct 14 00:28 f4gs13fs0y9ckrjjo78oid7hkv61k2wq.zip
drwxrwxr-x 3 root root       4096 Oct 14 01:10 granuleCell63XStart
drwxr-xr-x 1 root root       4096 Aug 27 16:17 sample_data
drwxrwxr-x 5 root root       4096 Oct 14 00:40 ShuTuUbuntu
-rw-r--r-- 1 root root  295628950 Oct 14 00:39 wszpq3r6hwgoxo8eq0vb1k82fwrabycl.zip
------------
/content/granuleCell63XStart/granule-1.Proj.tif
/content/granuleCell63XStart/granule-1.tif
/content/granuleCell63XStart/granule-1.tif.linkedPoints.dat
/content/granuleCell63XStart/granule-2.Proj.tif
/content/granuleCell63XStart/granule-2.tif
/content/granuleCell63XStart/granule-2.tif.linkedPoints.dat
/content/granuleCell63XStart/granule-3.Proj.tif
/content/granuleCell63XStart/granule-3.tif
/content/granuleCell63XStart/granule-3.tif.linkedPoints.dat
/content/granuleCell63XStart/granule-4.Proj.tif
/content/granuleCell63XStart/granule-4.tif
/content/granuleCell63XStart/granule-4.tif.linkedPoints.dat
/conten

In [0]:
# Optionally, download that newly created SWC before it gets thrown away after a max of 12 hours.
from google.colab import files
files.download('/content/granuleCell63XStart/granule-.auto.swc') 